In [ ]:
import os
import json

from library_dicom.dicom_processor.model.Series import Series
from library_dicom.dicom_processor.model.SeriesPT import SeriesPT
from library_dicom.dicom_processor.tools.folders import get_series_path, write_json_file, remove_bi_file
from library_dicom.dicom_processor.tools.series import get_series_object


In [ ]:
batch_path = []
for i in range(0,3):
    batch_path.append('/media/salim/DD 2To/AHL2011/AHLGroupe'+str(i))

In [ ]:
series_paths = []
for input_path in batch_path:
    series_paths_current = get_series_path(input_path)
    series_paths = series_paths + series_paths_current
export_folder = '/media/salim/DD 2To/Json Ahl'

In [ ]:
not_image_folder=[]
non_valid_series=[]
non_attenuation_corrected=[]
empty_folders = []

for serie_path in series_paths:
    if(len(os.listdir(serie_path)) == 0):
        empty_folders.append(serie_path)
        continue
    dicom_serie = get_series_object(serie_path)
    dicomsInfo = dicom_serie.get_series_details()
    try:
        json_string = json.dumps(dicomsInfo)
    except Exception as err:
        print(err)
        print(serie_path)
        print(dicomsInfo)
    remove_bi_file(serie_path)
    if(dicom_serie.is_image_modality() == False):
        not_image_folder.append(serie_path)
    if(dicom_serie.is_series_valid() == False):
        non_valid_series.append(serie_path)
    if(type(dicom_serie) == SeriesPT and dicom_serie.is_corrected_attenuation() == False):
        non_attenuation_corrected.append(serie_path)
    write_json_file(export_folder, dicomsInfo['study']['StudyInstanceUID'], json_string)
    

In [ ]:
print(not_image_folder)
print(non_valid_series)
print(non_attenuation_corrected)
print(empty_folders)

In [ ]:
import shutil
for image_folder in not_image_folder:
    shutil.rmtree(image_folder)

for empty_folder in empty_folders:
    shutil.rmtree(empty_folder)

for non_attenuation in non_attenuation_corrected:
    shutil.rmtree(non_attenuation)